In [7]:
"""
Converts output of MediaPipe (.json) to a .txt file using all 33 pose keypoints.
(Adapted from a script originally for OpenPose output)
creat_db.py is required to be run after this script to merge all text files into format required for RNN for Human Activity Recognition - (Now 33*2D Pose Input)

Original OpenPose script by Stuart Eiffert 13/12/2017
Modifications for MediaPipe (33 points) by Your Name/AI 2025/05/28

All code is provided under the MIT License (assuming original license applies)

"""

import json
from pprint import pprint # Keep for debugging if needed
import glob, os
import numpy as np # For handling missing keypoints

# --- Configuration based on your MediaPipe script ---
data_path = os.path.join(os.getcwd(), r'MP_Data_JSON')
#activity_list = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move'])
activity_list = np.array(['ageru', 'understand', 'annsinnsuru' , 'heavy'])
num_videos_per_action =10 #100 

fixed_cluster_num = 1
fixed_camera_num = 1
fixed_repetition_num = 1

# Number of keypoints from MediaPipe Pose (33 landmarks)
NUM_KEYPOINTS_PER_HAND = 21
NUM_HANDS = 2
NUM_OUTPUT_VALUES = NUM_KEYPOINTS_PER_HAND * NUM_HANDS * 2 # 21点 * 2手 * 2次元(x, y) = 84

# Loop through actions (activities) and videos (subjects)
for activity_idx, activity_name in enumerate(activity_list):
    for video_num_loop in range(1, num_videos_per_action + 1):

        subject_num_str = str(video_num_loop)
        if video_num_loop < 10:
            subject_num_str = "0" + subject_num_str

        activity_num_str = str(activity_idx + 1)
        if activity_idx + 1 < 10:
             activity_num_str = "0" + activity_num_str

        frame_set = f"l0{fixed_cluster_num}_c0{fixed_camera_num}_s{subject_num_str}_a{activity_num_str}_r0{fixed_repetition_num}"
        current_video_json_path = os.path.join(data_path, activity_name, str(video_num_loop))

        if not os.path.isdir(current_video_json_path):
            print(f"Warning: Directory not found, skipping: {current_video_json_path}")
            continue

        os.chdir(current_video_json_path)

        kps_all_frames = []

        json_files = sorted(glob.glob("*.json"), key=lambda x: int(os.path.splitext(x)[0]))

        for file_name in json_files:
            with open(file_name) as data_file:
                data = json.load(data_file)

                frame_kps_hands = []

                # 左手のキーポイントを抽出 (x, yのみ) - 21点 * 2 = 42値
                left_hand_landmarks = data.get("left_hand", [])
                if left_hand_landmarks:
                    for landmark in left_hand_landmarks:
                        frame_kps_hands.append(landmark.get('x', 0.0))
                        frame_kps_hands.append(landmark.get('y', 0.0))
                else:
                # 手が検出されなかった場合はゼロで埋める
                    frame_kps_hands.extend([0.0] * (NUM_KEYPOINTS_PER_HAND * 2))

        # 右手のキーポイントを抽出 (x, yのみ) - 21点 * 2 = 42値
                right_hand_landmarks = data.get("right_hand", [])
                if right_hand_landmarks:
                    for landmark in right_hand_landmarks:
                        frame_kps_hands.append(landmark.get('x', 0.0))
                        frame_kps_hands.append(landmark.get('y', 0.0))
                else:
                # 手が検出されなかった場合はゼロで埋める
                    frame_kps_hands.extend([0.0] * (NUM_KEYPOINTS_PER_HAND * 2))

                kps_all_frames.append(frame_kps_hands)

        output_dir_for_action = os.path.join(data_path, activity_name)
        if not os.path.isdir(output_dir_for_action):
             os.makedirs(output_dir_for_action)

        output_file_name = f"{activity_name}{frame_set}_mp33.txt" # Added _mp33 to distinguish
        output_file_path = os.path.join(output_dir_for_action, output_file_name)

        with open(output_file_path, "w") as text_file:
            for frame_data in kps_all_frames:
                if len(frame_data) != NUM_OUTPUT_VALUES:
                    print(f"Error: Frame data for {output_file_name} has {len(frame_data)} values, expected {NUM_OUTPUT_VALUES}. Skipping frame or writing as is.")
                    # Decide how to handle this: skip, pad, or write as is. For now, writing as is.
                
                for i, val in enumerate(frame_data):
                    text_file.write('{}'.format(val))
                    # Adjust comma separation for NUM_OUTPUT_VALUES
                    if i < (NUM_OUTPUT_VALUES - 1):
                        text_file.write(',')
                text_file.write('\n')
        print(f"Successfully wrote MediaPipe 42-point data for {activity_name} video {video_num_loop} to {output_file_path}")